# Machine Translation with RNNs

## Machine Learning 410
### Steve Elston

## Statistical Machine Translation

[Statistical machine translation](https://en.wikipedia.org/wiki/Statistical_machine_translation) (SMT) has a long history in AI, with work appearing in the early 1950s. The goal of SMT is to find the most probable output or **target sequence** of words in one language, $\mathbf{y} = y_1, y_2, \ldots, y_m$ given the **input sequence**, $\mathbf{x} = x_1, x_2, \ldots, x_n$, in another language. Using Bayes theorem, we can characterize the goal of STM as maximizing the conditional probability:

$$p(\mathbf{y} | \mathbf{x}) \propto p(\mathbf{x} | \mathbf{y}) p(\mathbf{y})$$

Where:   
$p(\mathbf{x} | \mathbf{y})$ is the **translation model**, giving the likelihood for the sequence $\mathbf{x}$ given the sequence $\mathbf{y}$.    
$p(\mathbf{y})$ is the language model, or probability that the sequence $\mathbf{y}$ occurs in the target language.   



### Log-Linear Model for STM

It is common practice in STM to maximize a log-linear model. Given a set of weights for $N$ possible output sequences, $\mathbf{w} = w_1, w_2, \ldots, w_N$, used to maximize the log-likelihood gives:  

$$log p(\mathbf{y} | \mathbf{x}) = \sum^N_{n=1} w_n p(\mathbf{y} | \mathbf{x}) + log Z(\mathbf{x})$$ 

Where, $Z(\mathbf{x})$ is the normalization constant, and is independent of the weights. Thus, this constant need not be computed to perform the maximization. 

The optimization of the log-likelihood can be done by maximizing the BLEU; see below. 

The log-likelihood can be computed for several types of sequences. For example, with **Phrases** the probability $p(\mathbf{x} | \mathbf{y})$ is factored into probabilities of matching phrases in the source and target sequence. 


### BLEU score

A commonly used metric for the quality of machine translations is the **Bilingual Translation Understudy** or **BLEU score**. The BLEU score compares the quality of a machine translated sequence to that of translations created by human experts. Machine translations with higher BLEU scores are considered better.   

For a sequence of length N the BLEU is computed:

$$BLEU = \{\prod_{i=1}^N P(i) \}^{\frac{1}{N}}$$   

Where;  

$$P(i) = \frac{Matched(i)}{H(i)}$$

Here, $H(i)$ is the number of number of *i-gram* tuples in each hypothesis. Examples for hypothesis of lengths, $n$, are $H(1) = n$,  $H(2) = n-1$, and  $H(3) = n-2$. 

There are multiple hypothesis to be tested. Let $t_i$ be an *i-gram* tuple of hypothesis $h$, and $C_h(t_i)$ is the number of time $t_i$ occurs in the hypothesis being tested, and $C_{hj}(t_i)$ is the number of times $t_i$ occurs in the $j$-th reference hypothesis, then:

$$Matched(i) = \sum_{t_i} min \{ C_h(t_i), \underset{j}{max} C_{hj}(t_i) \}$$

The above formulation of BLEU has a well known problem, in that the score is dependent on the length of the sequence. A widely used correction in the **brevity penalty**, $\rho$. The adjusted BELU is then:   

$$BLEU_\rho = exp \big( min( 0, \frac{n-L}{n} ) \big) \{\prod_{i=1}^N P(i) \}^{\frac{1}{N}}$$  

Where, $n$ is the length of the hypothesis and $L$ is the length of the reference sequence. 

## Neural Machine Translation   

We need to translate a variable length **input sequence**, $x_1, x_2, \ldots, x_n$, to a **target sequence**, $y_1, y_2, \ldots, y_m$. The neural machine translation (NMT) system learns to produce the target sequence given the input sequence. The NMT system has two components.    

First, an encoder computes a hidden representation, $\mathbf{h} = h_1, h_2, \ldots, h_n$, given the source sequence:

$$h_t = f(h_{t-1}, x_t)$$   

Where $f()$ is the activation function. The context is 

$$h_1, h_2, \ldots, h_n = Encoder_{RNN} (x_1, x_2, \ldots, x_n)$$

Second, a decoder generates target words based on the conditional probability given earlier words in the sequence: 

$$p(y_1, y_2, \ldots, y_m) = \prod_{t=1}^m p(y_t | \{ y, c, h \}_{\le m})$$

Or, 

$$log p(y_1, y_2, \ldots, y_m) = \sum_{t=1}^m log p(y_t |  \{ y, c, h \}_{\le m})$$

 

### Hidden States for NMT

NMT architectures primarily use layers of LSTM recurrent units. The states of these recurrent units are hidden from the input and output layers. Recall the update relationship:

$$h_t = f(h_{t-1}, x_t)$$ 

For NMT the first two recurrent layers (with respect to the input) comprise a **bidirectional recurrent neural network (BiRNN)**. Given the input sequence, $\mathbf{x} = x_1, x_2, \ldots, x_n$, the forward RNN layer is a sequence of states over $T$ time steps:   

$$\{ \overrightarrow{h_1}, \overrightarrow{h_2}, \ldots, \overrightarrow{h_T} \}$$

The reverse RNN layer is a sequence of states in reverse order to the input sequence:  

$$\{ \overleftarrow{h_T}, \overleftarrow{h_{T-1}}, \ldots, \overleftarrow{h_1} \}$$

The context vector at time $t$ can be expressed:    

$$c_t = \big[ \overrightarrow{h_t}; \overleftarrow{h_t} \big]$$

## Attention Mechanism

The use of fixed **context vector**, $c$, limits the representation of the sequence. This can cause difficulty, particularly for long sequences. As an alternative, we can use a **context set**, comprised of context vectors: 
$$\mathscr{c} = \{c_1, c_2, \ldots, c_M \}$$ 

The components of the vector localize a particular aspect of the input sequence. The **attention mechanism** scores each component of the context as a function of the hidden state, $h$:  

$$e_i^t = f_{ATT}(h_{t-1},c_i, \{ \alpha_j^{t-1} \}_{j=1}^M)$$

The **attention weights**, $\alpha_j^{t-1}$, are the attention weights from the last time step. The attention weights are are computed from the normalized scores, $e_i^{t-1}$:

$$\alpha_i^{t} = \frac{exp(e^t_i)}{\sum_{j=1}^M exp(e^t_j)} $$  

The attention weights can be thought of as the probability that the decoder should **attend** by the decoder to each context, $c_i$.

The attention weights are used to update the context the 

$$e_i^t = \phi(\{ c_i^{t} \}_{i=1}^M, \{ \alpha_i^{t} \}_{i=1}^M)$$

One possible model is simply linear weighting of th context vectors:

$$e_i^t = \phi(\{ c_i^{t} \}_{i=1}^M, \{ \alpha_i^{t} \}_{i=1}^M)
= \sum_{i=1}^M \alpha_i c_i$$


For this linear weighted model, the attention weights can be found as a **maximum likelihood problem**. The maximum likelihood function has parameters $\Theta$, and for $N$ training samples can be written:

$$\mathcal{L} \big(D, \Theta \big) = \frac{1}{N} \sum_{n=1}^N log\ p(y^n | x^n, \Theta)$$

Where, the training data of $N$ input and output sequences can be expressed: $D = \{(x_1,y_1), (x_2,y_2), \ldots, (x_N,y_N) \}$.

These attention weights are computed using a deep neural network as the function approximator. Since the model is differentiable, the neural network can be trained using **stochastic gradient decent** with **backpropagation** to compute these attention weights.   

## Decoder

The decoder uses a **beam search** to find the most probable sequence. Typical attention scores are biased to favor shorter sequences since sums of the log-probabilities will generally be lower for short sequences. 

Thus, a modified score likelihood function is used to adjust for output sequence length:

\begin{align}
\mathcal{L} \big(D, \Theta \big) &= \frac{1}{lp(Y)}\frac{1}{N} \sum_{n=1}^N log\ p(y^n | x^n, \Theta) +cp(D) \\
lp(Y) &= \frac{(5 + |Y|)^\gamma}{(5 + 1)^\gamma} \\
cp(D) &= \beta\ \sum_{i = 1}^{|X|} log \big( min( \sum_{j = 1}^{|Y|} \alpha_{i,j}, 1.0 )\big)
\end{align}   

Here;    
$\gamma$ is a parameter that determines the strength of the normalization.   
$\beta$ is a parameter that determines the extent to which translations that fully cover the source sentence are favored. 

## Residual connections

Training deep LSTM RNNs can be problematic. The gradient vanishes after only a few layers. The solution is to use **residual connections**. 

To understand how this works, first consider the the update equations between the $i$-th and $i+1$-th layers for standard stacked LSTM models. The weight tensors are $\mathbf{W}^i$ and $\mathbf{W}^{i+1}$ respectively. Given hidden states $h^i$, LSTM memory $m^i$, and input $x^i$, we can write the update relations at time step $t$:   

\begin{align}
h^i_t, m_t^i &= LSTM_i(h^i_{t-1}, m^i_{t-1}, x^{i-1}_t, \mathbf{W}^{i})\\
x^i_t &= h^i_t \\
h^{i+1}_t, m_t^{i+1} &= LSTM_{i+1}(h^{i+1}_{t-1}, m^{i+1}_{t-1}, x^{i}_t, \mathbf{W}^{i+1})
\end{align}

With residual connections added, the update equations become:

\begin{align}
h^i_t, m_t^i &= LSTM_i(h^i_{t-1}, m^i_{t-1}, x^{i-1}_t, \mathbf{W}^{i})\\
x^i_t &= h^i_t +  x^{i-1}_t\\
h^{i+1}_t, m_t^{i+1} &= LSTM_{i+1}(h^{i+1}_{t-1}, m^{i+1}_{t-1}, x^{i}_t, \mathbf{W}^{i+1})
\end{align}

## Wordpiece Module 

The ................

#### Copyright 2019 Stephen F Elston. All rights reserved